In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from subprocess import run
from glob import glob
from sklearn.linear_model import LinearRegression

In [ ]:
rootdir="/Volumes/NIMH_SFIM/handwerkerd/ComplexMultiEcho1/Data"
os.chdir(rootdir)
numsbj=11
numruns=3
sbjcol=[]
runcol=[]
for sbjidx in range(numsbj):
    for runidx in range(numruns):
        sbjcol.append(f"sub-{(sbjidx+1):02}")
        runcol.append(runidx+1)

# Initialize the DataFrame with one run for each subject and run
CompVar = pd.DataFrame({"sbj": sbjcol, "run": runcol})


In [ ]:
pcamethods = ['kic', 'c65', 'c70', 'c75']
rowidx=0

CompVar[f'motion'] = None
for pmethod in pcamethods:
    CompVar[f'{pmethod} norm var'] = None
    CompVar[f'{pmethod} numcomp'] = None
    CompVar[f'{pmethod} varex'] = None

for sbjidx in range(numsbj):
    sbj = f"sub-{(sbjidx+1):02}"
    # Calculate mean motion per run
    motpd = pd.read_csv(f"{rootdir}/{sbj}/afniproc_orig/WNW/{sbj}.results/motion_{sbj}_enorm.1D")
    mot = np.squeeze(motpd.to_numpy())
    runlen = int(len(mot)/3)
    meanmot = np.array([(mot[0:runlen]).mean(), (mot[runlen:(2*runlen)]).mean(), (mot[(2*runlen):len(mot)]).mean()])

    for runidx in range(numruns):

        CompVar.loc[rowidx, f'motion'] = meanmot[runidx]
        
        for pmethod in pcamethods:
            tmpfile = glob(f"{rootdir}/{sbj}/afniproc_orig/WNW/{sbj}.results/tedana_{pmethod}_r0{runidx+1}/tedana_202?-??-*.tsv")
            # reading in normalized variance
            tmpcmd = run(["grep",
                "normalized variance explained using", tmpfile[0]
            ], capture_output=True)
            tmpstr = str(tmpcmd.stdout)

            tmp_preidx = tmpstr.find('with')
            tmp_postidx = tmpstr.find('normalized variance explained using')
            if tmp_preidx>0 and tmp_postidx<len(tmpstr):
                CompVar.loc[rowidx, f'{pmethod} norm var'] = float(tmpstr[(tmp_preidx+4):(tmp_postidx-2)])
            # reading in variance explained
            tmpcmd = run(["grep",
                "Variance explained by decomposition:", tmpfile[0]
            ], capture_output=True)
            tmpstr = str(tmpcmd.stdout)

            tmp_preidx = tmpstr.find("decomposition:")
            tmp_postidx = tmpstr.find('%')
            if tmp_preidx>0 and tmp_postidx<len(tmpstr):
                CompVar.loc[rowidx, f'{pmethod} varex'] = float(tmpstr[(tmp_preidx+14):(tmp_postidx)])


        # reading in number of components
        for pmethod in pcamethods:
            tmpcmd = run(["wc", "-l",
                        f"{rootdir}/{sbj}/afniproc_orig/WNW/{sbj}.results/tedana_{pmethod}_r0{runidx+1}/pca_metrics.tsv"
            ], capture_output=True)
            tmpstr = (tmpcmd.stdout).decode('utf-8')
            try:
                CompVar.loc[rowidx, f'{pmethod} numcomp'] = int(tmpstr.split("/")[0])-1
            except:
                CompVar.loc[rowidx, f'{pmethod} numcomp'] = None

        rowidx+=1
CompVar

In [ ]:
# CompVar.plot.scatter(x='kic numcomp', y='kic norm var')
markersize=1
ax1= CompVar.plot.scatter(x='kic norm var', y='kic varex', color='b', s=markersize)
CompVar.plot.scatter(x='c65 norm var', y='c65 varex', color='r',s=markersize, ax=ax1)
CompVar.plot.scatter(x='c70 norm var', y='c70 varex', color='g', s=markersize, ax=ax1)
CompVar.plot.scatter(x='c75 norm var', y='c75 varex', color='c', s=markersize, ax=ax1)
# ax1.plot(np.linspace(70, 80), np.linspace(70,80), color='k')
ax1.legend(["kic", 'c65', 'c70', 'c75'])
ax1.set_xlabel("Normalized Variance")
ax1.set_ylabel("Variance")


In [ ]:

markersize=1
ax1= CompVar.plot.scatter(x='motion', y='kic norm var', color='b', s=markersize)
CompVar.plot.scatter(x='motion', y='c65 norm var', color='r',s=markersize, ax=ax1)
CompVar.plot.scatter(x='motion', y='c70 norm var', color='g', s=markersize, ax=ax1)
CompVar.plot.scatter(x='motion', y='c75 norm var', color='c', s=markersize, ax=ax1)

model = LinearRegression()
model.fit((CompVar['motion'].to_numpy()).reshape(-1, 1), CompVar['kic norm var'].to_numpy())
x_new = np.linspace(0.03, 0.18, 50)
y_new = model.predict(x_new[:, np.newaxis])
ax1.plot(x_new, y_new, color='b', linewidth=0.5)

model = LinearRegression()
model.fit((CompVar['motion'].to_numpy()).reshape(-1, 1), CompVar['c65 norm var'].to_numpy())
x_new = np.linspace(0.03, 0.18, 50)
y_new = model.predict(x_new[:, np.newaxis])
ax1.plot(x_new, y_new, color='r', linewidth=0.5)

model = LinearRegression()
model.fit((CompVar['motion'].to_numpy()).reshape(-1, 1), CompVar['c70 norm var'].to_numpy())
x_new = np.linspace(0.03, 0.18, 50)
y_new = model.predict(x_new[:, np.newaxis])
ax1.plot(x_new, y_new, color='g', linewidth=0.5)

model = LinearRegression()
model.fit((CompVar['motion'].to_numpy()).reshape(-1, 1), CompVar['c75 norm var'].to_numpy())
x_new = np.linspace(0.03, 0.18, 50)
y_new = model.predict(x_new[:, np.newaxis])
ax1.plot(x_new, y_new, color='c', linewidth=0.5)

# ax1.plot(np.linspace(70, 80), np.linspace(70,80), color='k')
ax1.legend(["kic", 'c65', 'c70', 'c75'])
ax1.set_xlabel("motion")
ax1.set_ylabel("Normalized Variance")

In [ ]:
markersize=1
ax1= CompVar.plot.scatter(x='motion', y='kic varex', color='b', s=markersize)
CompVar.plot.scatter(x='motion', y='c65 varex', color='r',s=markersize, ax=ax1)
CompVar.plot.scatter(x='motion', y='c70 varex', color='g', s=markersize, ax=ax1)
CompVar.plot.scatter(x='motion', y='c75 varex', color='c', s=markersize, ax=ax1)
# ax1.plot(np.linspace(70, 80), np.linspace(70,80), color='k')
ax1.legend(["kic", 'c65', 'c70', 'c75'])
ax1.set_xlabel("motion")
ax1.set_ylabel("Variance")

In [ ]:
print(f"Mean normvar {CompVar['kic norm var'].mean()}")
print(f"Median normvar {CompVar['kic norm var'].median()}")
print(f"Mean var {CompVar['kic varex'].mean()}")
print(f"Median var {CompVar['kic varex'].median()}")

In [ ]:
motpd = pd.read_csv(f"{rootdir}/sub-01/afniproc_orig/WNW/sub-01.results/motion_sub-01_enorm.1D")
mot = np.squeeze(motpd.to_numpy())
runlen = int(len(mot)/3)
meanmot = np.array([(mot[0:runlen]).mean(), (mot[runlen:(2*runlen)]).mean(), (mot[(2*runlen):len(mot)]).mean()])
print(meanmot)

In [ ]:
mot